# Preparing Computable Matrices

In [24]:
import os 
import pandas as pd 
import numpy as np

# start at the project dir
if 'workflow' not in os.listdir():
    os.chdir('../../../')
    
# create the output dir
datadir = 'results/harmonized/'
outdir = 'results/processed/harmonized/'
os.makedirs(outdir, exist_ok=True)

## Load the datasets

In [30]:
# setting a list of assays
assays = {'plasma_cytokine_concentrations': 'results/harmonized/training_plasma_cytokine_concentrations_by_olink_wide.tsv',
          'pbmc_cell_frequency': 'results/harmonized/training_pbmc_cell_frequency_wide.tsv',
          'plasma_antibody_levels': 'results/harmonized/training_plasma_antibody_levels_wide.tsv',
          'pbmc_gene_expression': 'results/harmonized/training_pbmc_gene_expression_wide_tpm.tsv'}

In [31]:
# load the specimen/clinical data
wetlab_data = pd.read_table('results/harmonized/training_subject_specimen.tsv')
wetlab_data = clinical_data.loc[clinical_data.planned_day_relative_to_boost == 0]

In [34]:
for assay, fn in assays.items():
    
    print(assay)

    # load the main data
    main_data = pd.read_table(fn, index_col=0)
    
    # add clinical data
    temp_data = main_data.merge(wetlab_data, left_index=True, right_on='specimen_id')
    
    # aggregate sample data
    agg_data = temp_data.groupby('subject_id')[main_data.columns.tolist()].agg(np.mean)

    # save the agg data
    output_fn = os.path.join(outdir, 'training_{}.tsv'.format(assay))
    agg_data.to_csv(output_fn, sep='\t')
    
    break

plasma_cytokine_concentrations


In [35]:
agg_data

,P05231,P51671,P22301,P48061,P10145,P80098,P01133,P13232,P13500,O14625,...,P09919,P13725,P40933,P49771,P78380,Q16552,Q8NEV9_Q14213,Q969D9,Q96PD4,Q9P0M4
subject_id,,,,,,,,,,,,,,,,,,,,,
4,3.50762,8.90560,3.47677,1.16403,7.27428,2.34641,8.95158,7.35730,10.44714,10.23940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.60569,9.12917,4.15272,1.16403,6.73318,3.78857,9.37102,3.53769,10.70867,10.34610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,4.25658,8.61154,3.34559,1.16403,5.85119,2.73679,9.45854,3.95864,9.98010,9.54684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2.64932,9.12795,4.03578,1.16403,5.99654,3.14737,9.68254,3.79068,10.63419,9.93890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,3.36796,9.11674,3.35656,1.17249,7.88393,3.36224,9.99899,3.73671,10.86716,11.17246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2.60737,185.29580,2.71494,108.75897,13.58607,2.89993,117.15139,0.89886,187.75042,548.29880,...,99.32790,8.02942,9.41378,72.47608,42.14676,0.17247,6.70182,NaN,0.40086,10.33650
115,1.44516,191.28646,6.03487,113.54777,8.57138,1.82234,71.51467,0.68607,239.44299,381.56898,...,60.63100,4.13839,14.51970,98.48871,106.97073,0.54016,6.47633,NaN,0.03561,14.25357
116,2.64423,58.37848,3.15700,163.68818,1.77119,5.19359,18.49992,6.28777,162.54787,14.68916,...,428.83330,1.86987,8.19912,71.03037,125.19664,2.39660,7.60560,9.76336,7.28124,13.56425
